# using `Array.prototype.reduce()` to group by an object property

A [StackOverflow answer](https://stackoverflow.com/a/40774906/22944) from [Nina Scholz](https://stackoverflow.com/users/1447675/nina-scholz) clearly and excellently shows how to use `Array.prototype.reduce()` [📖 [docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Array/reduce)] to group by an object property:

In [1]:
const cars = [
    { make: 'audi', model: 'r8', year: '2012' },
    { make: 'audi', model: 'rs5', year: '2013' },
    { make: 'ford', model: 'mustang', year: '2012' },
    { make: 'ford', model: 'fusion', year: '2015' },
    { make: 'kia', model: 'optima', year: '2012' }
];

const result =
    cars.reduce((a, i) => {
        a[i.make] = a[i.make] ?? [];
        a[i.make].push(i);
        return a;
    },
    {});

result

{
  audi: [
    { make: 'audi', model: 'r8', year: '2012' },
    { make: 'audi', model: 'rs5', year: '2013' }
  ],
  ford: [
    { make: 'ford', model: 'mustang', year: '2012' },
    { make: 'ford', model: 'fusion', year: '2015' }
  ],
  kia: [ { make: 'kia', model: 'optima', year: '2012' } ]
}

Nina’s original answer was in pure JavaScript and I have made a few modernizations:

1. I have changed `r[a.make] || []` to `r[a.make] ?? []`—to clarify intent.
2. I have changed `Object.create(null)` to `{}`.

## making Nina’s answer generic

To make Nina’s answer more reusable, we can start by taking a peek at [a StackOverflow answer from 2020](https://stackoverflow.com/a/47385953/22944) and define a function called `groupByKey`:

In [2]:
function groupByKey(reducible: {}[], key: string) : {} {
    const initialValue = {};
    const result = reducible.reduce((a: {}, i: {}) => {
        const groupName: any = i[key];
        a[groupName] = a[groupName] ?? [];
        a[groupName].push(i);
        return a;
    },
    initialValue);

    return result;
}

groupByKey(cars, 'make')

{
  audi: [
    { make: 'audi', model: 'r8', year: '2012' },
    { make: 'audi', model: 'rs5', year: '2013' }
  ],
  ford: [
    { make: 'ford', model: 'mustang', year: '2012' },
    { make: 'ford', model: 'fusion', year: '2015' }
  ],
  kia: [ { make: 'kia', model: 'optima', year: '2012' } ]
}

The type annotations on `reducible.reduce` make clearer what is happening in both `reduce` operations: the anonymous object `a` is _accumulating_ object properties. This following set operation runs _for the first time_ it is saying, “Add a new property with the name `groupName` and set the value of this property to an empty array”:

```typescript
a[groupName] = a[groupName] ?? [];
```

When the above runs a second (or subsequent) time it is somewhat uselessly setting itself to itself (but at compile time it is helping to define the type of `a[groupName]` as `any[]`).

## the verbose way of inferring the type of `a[groupName]`

Sometimes terse code does not help the less experienced programmers on the team. We can go back to Nina’s `reduce` lambda and make it (slightly) easier to understand:

In [3]:
const result2 =
    cars.reduce((a, i) => {
        if(!a[i.make]) { a[i.make] = [] };
        a[i.make].push(i);
        return a;
    },
    {});

result2

{
  audi: [
    { make: 'audi', model: 'r8', year: '2012' },
    { make: 'audi', model: 'rs5', year: '2013' }
  ],
  ford: [
    { make: 'ford', model: 'mustang', year: '2012' },
    { make: 'ford', model: 'fusion', year: '2015' }
  ],
  kia: [ { make: 'kia', model: 'optima', year: '2012' } ]
}

## additional reading

- “[Array Grouping in JavaScript: `array.groupBy()`](https://dmitripavlutin.com/javascript-array-group/)” by Dmitri Pavlutin

## <!---->

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼
